# Hello world!

In [1]:
import pandas as pd

In [2]:
pd.read_csv(r"raw/IndovestDKG_FULL.csv").head()

,subject,subject_type,relation,object,object_type,date
0,pt manulife aset manajemen indonesia,PERUSAHAAN,Mengumumkan,diversifikasi investasi,KONSEP,2025-03-07 21:18:00
1,ihsg,INDIKATOR_EKONOMI,Menghasilkan,pertumbuhan tahunan tertinggi,INDIKATOR_EKONOMI,2025-03-07 21:18:00
2,strategi diversifikasi,KONSEP,Meningkatkan,peluang return,KONSEP,2025-03-07 21:18:00
3,investor,ORANG,Mengendalikan,berbagai jenis aset investasi,KONSEP,2025-03-07 21:18:00
4,dimas ardinugraha,ORANG,Mempengaruhi,strategi diversifikasi,KONSEP,2025-03-07 21:18:00


In [3]:
full = pd.read_csv(r"raw/IndovestDKG_FULL.csv")
force_drop = pd.read_csv(r"raw/IndovestDKG_FORCEDROP.csv")